In [2]:
%pylab
from __future__ import print_function

%matplotlib inline

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from __future__ import print_function

import csv
from src import main, utils
import sys

projects = main.load_projects()
interests = ['argouml v0.26.2 method', 'mucommander v0.8.5 method']

p = list()
for project in projects:
    if ' '.join([project.name, project.version, project.level]) not in interests:
        continue
    p.append(project)

In [4]:
project = p[0]
print(project)
repos = main.load_repos(project)
goldsets = main.load_goldsets(project)

queries = main.create_queries(project)
snapshot = main.create_release_corpus(project, repos)
changesets = main.create_corpus(project, repos, main.ChangesetCorpus, use_level=False)

snapshot_lda, _  = main.create_lda_model(project, snapshot, None, "Release", use_level=True)
changeset_lda, _ = main.create_lda_model(project, changesets, None, "Changeset", use_level=False)

Project(name='argouml', printable_name='ArgoUML', version='v0.26.2', level='method', ref='refs/tags/VERSION_0_26_2', alpha=None, eta=None, passes=5, iterations=1000, num_topics=500, src_url='http://argouml-downloads.tigris.org/nonav/argouml-0.26.2/ArgoUML-0.26.2-src.zip', data_path='data/argouml/', full_path='data/argouml/v0.26.2/', src_path='data/argouml/v0.26.2/src')


In [5]:
q = dict()
for query, topics in main.get_topics(snapshot_lda, queries, by_ids=["5258", "5088"], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    q[query[0]] = topics

qc = dict()
for query, topics in main.get_topics(changeset_lda, queries, by_ids=["5258", "5088"], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    qc[query[0]] = topics
g5258 = ["org.argouml.ui.explorer.rules.GoClassifierToInstance.getRuleName()",
         "org.argouml.ui.explorer.rules.GoClassifierToInstance.getRuleName()"]
g5088 = ["org.argouml.model.mdr.XmiWriterMDRImpl.write()",
         "org.argouml.model.AbstractActivityGraphsHelperDecorator.setParameters(Object,Collection)"]
 
q5258 = dict()
for query, topics in main.get_topics(snapshot_lda, snapshot, by_ids=[g5258[0]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    q5258[query[0]] = topics
#print(q5258)

qc5258 = dict()
for query, topics in main.get_topics(changeset_lda, snapshot, by_ids=[g5258[1]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    qc5258[query[0]] = topics
#print(q5258)

q5088 = dict()
for query, topics in main.get_topics(snapshot_lda, snapshot, by_ids=[g5088[0]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    q5088[query[0]] = topics
#print(q5088)

qc5088 = dict()
for query, topics in main.get_topics(changeset_lda, snapshot, by_ids=[g5088[1]], full=False):
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    qc5088[query[0]] = topics
#print(qc5088)

In [6]:
def printer(q, model):
    for qid, topics in q.items():
        print(qid, "num topics:", len(topics))
        for t in topics:
            if len(topics) <= 3 or t[1] >= 0.1:
                print("    ", t[0], t[1])
                for word in model.show_topic(t[0]):
                    if word[0]>= 0.05:
                        print("        {0} {1}".format(*word))

# Ranks
- argouml 5258
    - snapshot 1 org.argouml.ui.explorer.rules.GoClassifierToInstance.getRuleName()
    - changeset 8138 org.argouml.ui.explorer.rules.GoClassifierToInstance.getRuleName()
- argouml 5088
    - changeset 1 org.argouml.model.AbstractActivityGraphsHelperDecorator.setParameters(Object,Collection)
    - snapshot 124 org.argouml.model.mdr.XmiWriterMDRImpl.write()

# Query words (preprocessed)

In [7]:
queries.metadata = True
qs = list(filter(lambda x: x[1][0] in ["5258", "5088"], queries))

for query, metadata in qs:
    doc = sorted(query, key=lambda x: x[1], reverse=True)
    words = [ ( freq, queries.id2word[wid] ) for wid, freq in doc ]
    print(metadata[0], "num words:", len(words))
    for word in words:
        print("        {1} ({0})".format(*word))

5088 num words: 49
        profiles (4)
        xmi (3)
        user (3)
        profile (3)
        write (3)
        save (2)
        files (2)
        defined (2)
        loaded (2)
        impl (2)
        mdr (2)
        writer (2)
        models (2)
        able (1)
        available (1)
        implemented (1)
        file (1)
        model (1)
        issue (1)
        release (1)
        aren (1)
        using (1)
        isn (1)
        zargo (1)
        creating (1)
        removed (1)
        configuration (1)
        removing (1)
        empty (1)
        won (1)
        seams (1)
        persister (1)
        usage (1)
        simply (1)
        written (1)
        depend (1)
        due (1)
        configured (1)
        deeper (1)
        directories (1)
        experimentally (1)
        flag (1)
        functionality (1)
        persist (1)
        prevents (1)
        tackle (1)
        unassigned (1)
        wasn (1)
        writing (1)
5258 num words: 6
        nam

# Query topics

In [8]:
print("Snapshot")
printer(q, snapshot_lda)

print()
print("Changeset")
printer(qc, changeset_lda)

Snapshot
5258 num topics: 3
     194 0.583261328174
        0.178309175492 rule
        0.121059499556 perspective
        0.0762293638816 name
        0.0690858822929 rules
        0.0503684867632 explorer
     226 0.188608428249
        0.1856541011 machine
        0.11777883397 object
        0.102723364576 handle
        0.0773127578072 collaboration
        0.0647654447602 expr
        0.0556342824257 represented
        0.0545723769322 set
     464 0.145296910244
        0.388703354048 classifier
        0.0789009067262 object
5088 num topics: 18
     281 0.191849937158
        0.0969630906947 profiles
        0.0777460707084 profile
     283 0.132888424299
        0.0805840860895 writer
        0.0625096636339 component
        0.0515641337096 file
        0.0505557374166 xmi
     38 0.11550899696
        0.0837400317472 file
        0.052783578714 chooser

Changeset
5258 num topics: 2
     250 0.480647755462
        0.14981641216 role
        0.0626290395827 classifier
     365

# Issue 5258 top method topics

In [9]:
print("Snapshot")
printer(q5258, snapshot_lda)

print()
print("Changeset")
printer(qc5258, changeset_lda)

Snapshot
org.argouml.ui.explorer.rules.GoClassifierToInstance.getRuleName() num topics: 3
     194 0.823015472673
        0.178309175492 rule
        0.121059499556 perspective
        0.0762293638816 name
        0.0690858822929 rules
        0.0503684867632 explorer
     226 0.0685924095711
        0.1856541011 machine
        0.11777883397 object
        0.102723364576 handle
        0.0773127578072 collaboration
        0.0647654447602 expr
        0.0556342824257 represented
        0.0545723769322 set
     464 0.0498912641392
        0.388703354048 classifier
        0.0789009067262 object

Changeset
org.argouml.ui.explorer.rules.GoClassifierToInstance.getRuleName() num topics: 2
     432 0.76531764969
        0.0893536710693 jar
        0.0678621781371 argouml
        0.0517429904434 org
     332 0.176094115016
        0.0834646177049 rule
        0.0673931412026 perspective


# Issue 5088 top method topics

In [10]:
print()
print("Snapshot")
printer(q5088, snapshot_lda)

print()
print("Changeset")
printer(qc5088, changeset_lda)


Snapshot
org.argouml.model.mdr.XmiWriterMDRImpl.write() num topics: 6
     283 0.559318918067
        0.0805840860895 writer
        0.0625096636339 component
        0.0515641337096 file
        0.0505557374166 xmi
     82 0.156183575621
        0.112620395345 xmi
        0.100644430345 input
        0.0743053978246 exception
        0.0681277303619 stream
        0.0663342788556 reader
        0.0647220093709 file
     41 0.117419979701
        0.116364685128 exception
        0.113558278325 object
        0.0930610719178 invalid
        0.0875901359546 handle
        0.0700790042915 element

Changeset
org.argouml.model.AbstractActivityGraphsHelperDecorator.setParameters(Object,Collection) num topics: 2
     481 0.544616865511
        0.22107180032 object
        0.0919750214017 lang
        0.0766887196664 java
        0.0726775483298 model
     22 0.384240277346
        0.112239881206 set
        0.0783031934781 parameter
